In [1]:
import pandas as pd
import re
from datetime import datetime
import pytz
import os



In [2]:
# Step No. 1: Data Extraction
try:
    # Load the data from the local CSV file
    file_path = r'C:\Users\N I T R O   5\Documents\Bootcamps\Purwadhika\Capstone\data_sources\data_sources\data_reqruitment\data_requirements.csv'
    data = pd.read_csv(file_path)
    print("Step No. 1: Data Extraction")
    print("-" * 80)
except Exception as e:
    print(f"Step No. 1: Data Extraction Failed. Error: {e}")
    print("-" * 80)



Step No. 1: Data Extraction
--------------------------------------------------------------------------------


In [3]:
# Step No. 2: Data Transformation
try:
    # Remove unnecessary index column if exists
    if 'Unnamed: 0' in data.columns:
        data.drop(columns=['Unnamed: 0'], inplace=True)
        
    # Remove the job description column
    if 'job_description' in data.columns:
        data.drop(columns=['job_description'], inplace=True)

    # Remove rows with any empty cells
    data_cleaned = data.dropna().copy()

    # Standardize salary_estimate column (keeping as-is)
    def standardize_salary(salary):
        try:
            # Remove (est.) and extract the numeric part
            salary_value = re.sub(r'[\(\)est.]', '', salary).strip()
            salary_value = re.findall(r'[\d,]+', salary_value)[0]
            return float(salary_value.replace(',', ''))
        except:
            return None

    data_cleaned.loc[:, 'salary_estimate'] = data_cleaned['salary_estimate'].apply(standardize_salary)

    # Skip salary_numeric transformation as requested

    # Clean up company_type column to remove 'Company - ' prefix
    # Retain only the company type without the 'Company - ' prefix
    data_cleaned.loc[:, 'company_type'] = data_cleaned['company_type'].str.replace(r'^Company - ', '', regex=True)

    # Remove rows where company_size contains a date-like string
    date_pattern = r'\d{4}'
    data_cleaned = data_cleaned[~data_cleaned['company_size'].str.contains(date_pattern, na=False)]

    # Shorten company_size column
    def shorten_company_size(size):
        try:
            return size.replace('Employees', '').replace('to', '-').strip()
        except:
            return size

    data_cleaned.loc[:, 'company_size'] = data_cleaned['company_size'].apply(shorten_company_size)

    # Convert date column to 'Asia/Jakarta' timezone
    def convert_to_jakarta_timezone(date_str):
        try:
            date_obj = datetime.fromisoformat(date_str)
            jakarta_timezone = pytz.timezone('Asia/Jakarta')
            date_obj_jakarta = date_obj.astimezone(jakarta_timezone)
            return date_obj_jakarta
        except Exception:
            return None

    # Apply date transformation
    data_cleaned.loc[:, 'dates_jakarta'] = data_cleaned['dates'].apply(convert_to_jakarta_timezone)

    print("Step No. 2: Data Transformation")
    print("-" * 80)
except Exception as e:
    print(f"Step No. 2: Data Transformation Failed. Error: {e}")
    print("-" * 80)



C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_30564\381404926.py:24: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_cleaned.loc[:, 'salary_estimate'] = data_cleaned['salary_estimate'].apply(standardize_salary)


Step No. 2: Data Transformation
--------------------------------------------------------------------------------


In [4]:
# Step No. 3: Data Demography Analysis
try:
    # Top 3 companies with the highest rating
    top_3_companies = data_cleaned.sort_values(by='company_rating', ascending=False).head(3)[['company', 'company_rating']]
    print("Step No. 3: Top 3 Companies with Highest Rating")
    print(top_3_companies)
    print("-" * 80)

    # Top 5 Locations with the Most Jobs
    top_5_locations = data_cleaned['location'].value_counts().head(5)
    print("Step No. 4: Top 5 Locations with Most Jobs")
    print(top_5_locations)
    print("-" * 80)

    # Sum of each job title
    job_title_counts = data_cleaned['job_title'].value_counts()
    print("Step No. 5: Job Title Counts")
    print(job_title_counts)
    print("-" * 80)

    # Job with the highest salary estimation
    if 'salary_numeric' in data_cleaned.columns:
        top_salary_job = data_cleaned.loc[data_cleaned['salary_numeric'].idxmax()][['job_title', 'salary_numeric']]
        print("Step No. 6: Job with the Highest Salary Estimation")
        print(top_salary_job)
    else:
        print("Step No. 6: salary_numeric column is missing.")
    print("-" * 80)

    # Top 3 company sizes
    top_3_company_sizes = data_cleaned['company_size'].value_counts().head(3)
    print("Step No. 7: Top 3 Company Sizes")
    print(top_3_company_sizes)
    print("-" * 80)

    # Count of each company size
    company_size_count = data_cleaned['company_size'].value_counts()
    print("Step No. 8: Company Size Count")
    print(company_size_count)
    print("-" * 80)

    # Count of each company type
    company_type_count = data_cleaned['company_type'].value_counts()
    print("Step No. 9: Company Type Count")
    print(company_type_count)
    print("-" * 80)

    # Count of each company sector
    company_sector_count = data_cleaned['company_sector'].value_counts()
    print("Step No. 10: Company Sector Count")
    print(company_sector_count)
    print("-" * 80)

    # Company with the biggest revenue
    if 'company_revenue_standard' not in data_cleaned.columns:
        data_cleaned['company_revenue_standard'] = data_cleaned['company_revenue'].apply(lambda x: re.sub(r'[^0-9]', '', str(x)))
    biggest_revenue_company = data_cleaned.sort_values(by='company_revenue_standard', ascending=False).iloc[0][['company', 'company_revenue']]
    print("Step No. 11: Company with the Biggest Revenue")
    print(biggest_revenue_company)
    print("-" * 80)

    # Count companies in revenue classification
    def classify_revenue(revenue):
        if revenue is None or revenue == '':
            return 'Unknown'
        revenue = int(revenue)
        if revenue < 1000:
            return '1-1 billion'
        elif 1000 <= revenue < 5000:
            return '1-5 billion'
        elif 5000 <= revenue < 10000:
            return '5-10 billion'
        else:
            return '10+ billion'

    data_cleaned['revenue_classification'] = data_cleaned['company_revenue_standard'].apply(classify_revenue)
    revenue_classification_count = data_cleaned['revenue_classification'].value_counts()
    print("Step No. 12: Revenue Classification Count")
    print(revenue_classification_count)
    print("-" * 80)

    # Top companies with the highest salary numeric (formatted in 10,000 format)
    if 'salary_numeric' in data_cleaned.columns:
        top_salary_companies = data_cleaned.sort_values(by='salary_numeric', ascending=False).head(10)[['company', 'salary_numeric']]
        top_salary_companies['salary_numeric'] = top_salary_companies['salary_numeric'].apply(lambda x: f"{x:,.0f}")
        print("Step No. 13: Top Companies with the Highest Salary Numeric")
        print(top_salary_companies)
    else:
        print("Step No. 13: salary_numeric column is missing.")
    print("-" * 80)

    # Smallest, biggest, average, and median salary
    if 'salary_numeric' in data_cleaned.columns:
        salary_min = data_cleaned['salary_numeric'].min()
        salary_max = data_cleaned['salary_numeric'].max()
        salary_avg = data_cleaned['salary_numeric'].mean()
        salary_median = data_cleaned['salary_numeric'].median()

        print(f"Step No. 14: Salary Statistics\n Smallest: {salary_min:,.0f}, Biggest: {salary_max:,.0f}, Average: {salary_avg:,.0f}, Median: {salary_median:,.0f}")
    else:
        print("Step No. 14: salary_numeric column is missing, unable to perform salary statistics.")
    print("-" * 80)

    print("Step No. 3: Data Demography Analysis")
    print("-" * 80)
except Exception as e:
    print(f"Step No. 3: Data Demography Analysis Failed. Error: {e}")
    print("-" * 80)



Step No. 3: Top 3 Companies with Highest Rating
                             company  company_rating
834         APPIC Solutions LLC\n5.0             5.0
495         Slide Insurance LLC\n5.0             5.0
322  Stonehenge Technology Labs\n5.0             5.0
--------------------------------------------------------------------------------
Step No. 4: Top 5 Locations with Most Jobs
Remote            41
New York, NY      14
Atlanta, GA       12
Washington, DC     9
Austin, TX         9
Name: location, dtype: int64
--------------------------------------------------------------------------------
Step No. 5: Job Title Counts
Data Engineer                            90
Senior Data Engineer                     34
Sr. Data Engineer                         9
AWS Data Engineer                         6
Azure Data Engineer                       6
                                         ..
Data Engineer - Remote US                 1
GEM Data Engineer                         1
Data Quality Enginee

In [5]:
# Step No. 4: Data Loading
try:
    # Save the cleaned and transformed data to a new CSV file
    output_directory = os.path.dirname(file_path)
    base_filename = "cleaned_transformed_data"
    output_file_path = os.path.join(output_directory, f"{base_filename}.csv")
    
    # Check if file exists, if yes, add a number to the filename
    file_number = 1
    while os.path.exists(output_file_path):
        output_file_path = os.path.join(output_directory, f"{base_filename}_{file_number}.csv")
        file_number += 1

    data_cleaned.to_csv(output_file_path, index=False)
    print(f"Step No. 4: Data successfully saved to {output_file_path}")
    print("-" * 80)
except PermissionError:
    print(f"Step No. 4: Permission Denied. Please check file permissions or if the file is open in another program.")
    print("-" * 80)
except Exception as e:
    print(f"Step No. 4: Data Saving Failed. Error: {e}")
    print("-" * 80)



Step No. 4: Data successfully saved to C:\Users\N I T R O   5\Documents\Bootcamps\Purwadhika\Capstone\data_sources\data_sources\data_reqruitment\cleaned_transformed_data_5.csv
--------------------------------------------------------------------------------
